In [1]:
import requests
import json
import pandas as pd
import collections

In [77]:
r = requests.get('http://dataservices.imf.org/RESsT/SDMX_JSON.svc/Dataflow/')

In [3]:
def DataflowMethod():
    r = requests.get('http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow/')
    IDs = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['KeyFamilyRef'].apply(lambda x: x['KeyFamilyID'])
    Texts = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['Name'].apply(lambda x: x['#text'])
    return(pd.DataFrame({'DatabaseID': IDs, 
                         'DatabaseText': Texts}))

In [4]:
def DataStructureMethod(database_id, check_query = False):
    
    if check_query:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None

    r = requests.get(''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/',database_id]))
    if not r.ok:
        return None

    rparsed = r.json()
    dim_code = pd.Series(rparsed['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']).apply(lambda x: x['@codelist']).tolist()
    dim_codedict = [x for x in rparsed['Structure']['CodeLists']['CodeList'] if x['@id'] in dim_code]
    dim_codedict = dict(zip(pd.Series(dim_codedict).apply(lambda x: x['@id']).tolist(), dim_codedict))
    for k in dim_codedict.keys():
        dim_codedict[k] = pd.DataFrame({
                'CodeValue' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['@value']),
                'CodeText' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['Description']['#text'])
            })
    
    return((dim_code, dim_codedict))

In [226]:
def CompactDataMethod(database_id, queryfilter=None, 
                      startdate='2001-01-01', enddate='2001-12-31',
                      checkquery = False, verbose=False):
    
    if checkquery:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None
        
    queryfilterstr = ''
    if len(queryfilter[1]) > 0 and len(queryfilter[0]) == len(queryfilter[1]):
        queryfilterstr = '.'.join(['+'.join(queryfilter[1][k]) for k in queryfilter[0]])
        
    APIstr = ''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/',
                      databaseID,'/',queryfilterstr,'?startPeriod=',startdate,'&endPeriod=',enddate])
    r = requests.get(APIstr)        
    
    if verbose:
        print('\nmaking API call:\n')
        print(APIstr)
        print('\n')
        
    if not r.ok:
        return None        

    rparsed = r.json()
    
    if not 'Series' in list(rparsed['CompactData']['DataSet'].keys()):
        print('no data available \n')
        return None

    return_df = [x for x in rparsed['CompactData']['DataSet']['Series'] if 'Obs' in x]
    for x in return_df:
        if isinstance(x['Obs'],dict):
            x['Obs'] = [x['Obs']]

    return_df = pd.concat([pd.DataFrame(x) for x in return_df])
    obs_keys = return_df['Obs'].apply(lambda x: [y for y in x.keys()]).tolist()
    obs_keys = set([val for sublist in obs_keys for val in sublist])

    for obs_key in obs_keys:
        return_df[obs_key] = return_df['Obs'].apply(lambda x: x[obs_key] if obs_key in x.keys() else None)

    del return_df['Obs']
    return_df = return_df.reset_index()       

    return(return_df)


In [230]:
queryfilter = (['CL_FREA','CL_AREA_IFS','CL_INDICATOR_IFS'], 
               {'CL_FREA':['Q','A'],
                'CL_AREA_IFS':['GR'],
                'CL_INDICATOR_IFS':['NGDP_EUR','NGDP_XDC']})
CompactDataMethod('IFS', queryfilter)

,index,@FREQ,@INDICATOR,@REF_AREA,@TIME_FORMAT,@UNIT_MULT,@OBS_VALUE,@OBS_STATUS,@TIME_PERIOD
0,0,Q,NGDP_EUR,GR,P3M,9,35.2366,K,2001-Q1
1,1,Q,NGDP_EUR,GR,P3M,9,36.7264,None,2001-Q2
2,2,Q,NGDP_EUR,GR,P3M,9,39.8428,None,2001-Q3
3,3,Q,NGDP_EUR,GR,P3M,9,40.3881,None,2001-Q4
4,0,A,NGDP_EUR,GR,P1Y,9,151.9872,K,2001


In [99]:
# teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q+A.GR.NGDP_EUR+NGDP_XDC?startPeriod=2001-01-01&endPeriod=2001-12-31'
teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A.GR.NGDP_EUR+NGDP_XDC?startPeriod=1990-01-01&endPeriod=2001-12-31'
# teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q+A.GR.NGDP_EsUR+NGDP_sXDC?startPeriod=2001-01-01&endPeriod=2001-12-31'

In [225]:
r = requests.get(teststr)        
rparsed = r.json()
return_df = [x for x in rparsed['CompactData']['DataSet']['Series'] if 'Obs' in x]
for x in return_df:
    if isinstance(x['Obs'],dict):
        x['Obs'] = [x['Obs']]
        
return_df = pd.concat([pd.DataFrame(x) for x in return_df])
obs_keys = return_df['Obs'].apply(lambda x: [y for y in x.keys()]).tolist()
obs_keys = set([val for sublist in obs_keys for val in sublist])

for obs_key in obs_keys:
    return_df[obs_key] = return_df['Obs'].apply(lambda x: x[obs_key] if obs_key in x.keys() else None)

del return_df['Obs']
return_df = return_df.reset_index()

@OBS_VALUE
@OBS_STATUS
@TIME_PERIOD


In [224]:
return_df.reset_index()

,index,@FREQ,@INDICATOR,@REF_AREA,@TIME_FORMAT,@UNIT_MULT,@OBS_VALUE,@OBS_STATUS,@TIME_PERIOD
0,0,A,NGDP_XDC,GR,P1Y,9,13327.0016925,None,1990
1,1,A,NGDP_XDC,GR,P1Y,9,16458.9985025,None,1991
2,2,A,NGDP_XDC,GR,P1Y,9,19026.9995425,None,1992
3,3,A,NGDP_XDC,GR,P1Y,9,21423.9986025,None,1993
4,4,A,NGDP_XDC,GR,P1Y,9,24295.999755,None,1994
5,5,A,NGDP_XDC,GR,P1Y,9,30515.97529,None,1995
6,6,A,NGDP_XDC,GR,P1Y,9,33528.7470825,None,1996
7,7,A,NGDP_XDC,GR,P1Y,9,37103.006725,None,1997
8,8,A,NGDP_XDC,GR,P1Y,9,40344.2582075,None,1998
9,9,A,NGDP_XDC,GR,P1Y,9,42987.432105,None,1999


In [208]:
return_df['Obs'].iloc[11]['@OBS_VALUE']

'151.9872'

In [163]:
return_df['Obs'].appl

0    [{'@OBS_VALUE': '13327.0016925', '@TIME_PERIOD...
1    [{'@OBS_VALUE': '151.9872', '@OBS_STATUS': 'K'...
Name: Obs, dtype: object

In [130]:
rparsed['CompactData']['DataSet']['Series'][0]

{'@FREQ': 'A',
 '@INDICATOR': 'NGDP_XDC',
 '@REF_AREA': 'GR',
 '@TIME_FORMAT': 'P1Y',
 '@UNIT_MULT': '9',
 'Obs': [{'@OBS_VALUE': '13327.0016925', '@TIME_PERIOD': '1990'},
  {'@OBS_VALUE': '16458.9985025', '@TIME_PERIOD': '1991'},
  {'@OBS_VALUE': '19026.9995425', '@TIME_PERIOD': '1992'},
  {'@OBS_VALUE': '21423.9986025', '@TIME_PERIOD': '1993'},
  {'@OBS_VALUE': '24295.999755', '@TIME_PERIOD': '1994'},
  {'@OBS_VALUE': '30515.97529', '@TIME_PERIOD': '1995'},
  {'@OBS_VALUE': '33528.7470825', '@TIME_PERIOD': '1996'},
  {'@OBS_VALUE': '37103.006725', '@TIME_PERIOD': '1997'},
  {'@OBS_VALUE': '40344.2582075', '@TIME_PERIOD': '1998'},
  {'@OBS_VALUE': '42987.432105', '@TIME_PERIOD': '1999'},
  {'@OBS_VALUE': '46437.75075', '@TIME_PERIOD': '2000'}]}

In [30]:
adict = {'a':1, 'b':2, 'c':3}

In [31]:
'a' in adict

True

In [76]:
adict.iteritems()

AttributeError: 'dict' object has no attribute 'iteritems'